## Import Modules/ Libs for Data Clean-up

In [1]:
import pandas as pd
import csv
import datetime as dt
from datetime import datetime
import numpy as np

## Import SQL-Alchemy for SQL-Lite Data Load

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy import func
import pathlib

## Data Clean-up Process

### DATABASE RAW_DF

In [3]:
# define path to csv file
path="Resources_hle\IMDb_movies.csv"

# convert csv to pandas df
raw_df = pd.read_csv(path, encoding="UTF-8", dtype={'usa_gross_income': "string", "worlwide_gross_income" : "string", "budget": "string"})

1. choose all cols to keep (netflix id, imdb id, title, ddmmyy, dd, mo,yr,duration, avg_vote, votes, reviews_from_users	reviews_from_critics, budget,usa_gross_icome, ww income, coutry, type, genre, actor, description)
[[""]]
2. break the date published col into ye, mo, dd
3. combine reviews
4. rename columns
5. load into SQL-pgadmin4
6. report
       a. where data found, how
       b. transforming process
       c. load SQL, pic of SQL

In [4]:
# review what imported
raw_df

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0429277,Zyzzyx Rd,Zyzzyx Rd,2006,2/24/2006,"Crime, Drama, Thriller",90,USA,English,John Penney,...,"Leo Grillo, Katherine Heigl, Tom Sizemore, Ric...",The family man accountant Grant travels to Las...,4.0,930,"$2,000,000",$30,$30,NaN,11.0,1.0
1,tt0070913,Che?,Che?,1972,12/8/1972,Comedy,114,"Italy, France, West Germany","English, Italian, French",Roman Polanski,...,"Marcello Mastroianni, Sydne Rome, Hugh Griffit...","During her Italian vacation, a young and beaut...",5.7,3256,<NA>,$64,<NA>,NaN,23.0,37.0
2,tt1986953,Storage 24,Storage 24,2012,6/29/2012,"Action, Horror, Mystery",87,UK,English,Johannes Roberts,...,"Noel Clarke, Colin O'Donoghue, Antonia Campbel...","In London, a military plane crashes leaving it...",4.4,6309,<NA>,$72,"$646,175",52.0,65.0,88.0
3,tt1865335,Confession of a Child of the Century,Confession of a Child of the Century,2012,8/29/2012,Drama,120,"France, Germany, UK",English,Sylvie Verheyde,...,"Charlotte Gainsbourg, Pete Doherty, August Die...","Paris, 1830: Octave, betrayed by his mistress,...",4.4,514,EUR 4000000,$74,"$146,155",NaN,4.0,23.0
4,tt4195920,Chicas paranoicas,Chicas paranoicas,2015,9/16/2016,Comedy,100,Spain,Spanish,Pedro del Santo,...,"Patricia Valley, Mairen Muñoz, Marta Mir Martí...",'Chicas Paranoicas' is the first Spanish comed...,8.0,169,<NA>,$78,$78,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9894098,Sathru,Sathru,2019,3/7/2019,Thriller,129,India,NaN,Naveen Nanjundan,...,"Srushti Dange, Kathir, Laguparan, Marimuthu, N...",A kidnapping gone wrong leads to mounting tens...,6.1,163,<NA>,<NA>,"$8,683",NaN,7.0,1.0
81269,tt9899880,Columbus,Columbus,2018,12/5/2018,"Comedy, Drama",82,Iran,"Persian, English",Hatef Alimardani,...,"Farhad Aslani, Majid Salehi, Saeed Poursamimi,...",A rich family are deciding to immigrate to the...,4.0,130,<NA>,<NA>,<NA>,NaN,NaN,13.0
81270,tt9903716,Jessie,Jessie,2019,3/15/2019,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,"Sritha Chandana, Pavani Gangireddy, Abhinav Go...","Set in an abandoned house, the film follows a ...",7.2,219,<NA>,<NA>,<NA>,NaN,21.0,NaN
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,3/8/2019,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,8.4,369,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [5]:
# describe the data    
raw_df.describe()

,year,duration,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics
count,81273.000000,81273.000000,81273.000000,8.127300e+04,12722.000000,74196.000000,70286.000000
mean,1993.007210,100.565981,5.926587,9.421771e+03,55.762695,43.753194,27.992758
std,23.992284,25.320189,1.243315,5.220245e+04,17.757453,159.903568,58.708764
min,1906.000000,40.000000,1.000000,9.900000e+01,1.000000,1.000000,1.000000
25%,1979.000000,88.000000,5.200000,2.060000e+02,43.000000,4.000000,3.000000
50%,2002.000000,96.000000,6.100000,4.950000e+02,56.000000,9.000000,8.000000
75%,2012.000000,108.000000,6.800000,1.865000e+03,69.000000,26.000000,24.000000
max,2019.000000,3360.000000,10.000000,2.159628e+06,100.000000,8302.000000,987.000000


In [6]:
# extract day, month, year from the date_published columns
raw_df["date_published"] = pd.to_datetime(raw_df['date_published'])
raw_df['day'], raw_df['month'], raw_df['year']  = raw_df['date_published'].dt.day, raw_df['date_published'].dt.month, raw_df['date_published'].dt.year

In [7]:
# print out current columns with template to create a dictionary for columns rename 
for col in raw_df.columns:
    print(f'"{col}" : "",')

"imdb_title_id" : "",
"title" : "",
"original_title" : "",
"year" : "",
"date_published" : "",
"genre" : "",
"duration" : "",
"country" : "",
"language" : "",
"director" : "",
"writer" : "",
"production_company" : "",
"actors" : "",
"description" : "",
"avg_vote" : "",
"votes" : "",
"budget" : "",
"usa_gross_income" : "",
"worlwide_gross_income" : "",
"metascore" : "",
"reviews_from_users" : "",
"reviews_from_critics" : "",
"day" : "",
"month" : "",


In [8]:
# define what will be renamed
cols = {
"imdb_title_id" : "id",
"production_company" : "prod_co",
"avg_vote" : "user_rating",
"votes" : "vote_num",
"worlwide_gross_income" : "worldwide_gross_income",
"metascore" : "web_rating",
    }

# process new col names
raw_df.rename(columns=cols, inplace=True)
raw_df

,id,title,original_title,year,date_published,genre,duration,country,language,director,...,user_rating,vote_num,budget,usa_gross_income,worldwide_gross_income,web_rating,reviews_from_users,reviews_from_critics,day,month
0,tt0429277,Zyzzyx Rd,Zyzzyx Rd,2006,2006-02-24,"Crime, Drama, Thriller",90,USA,English,John Penney,...,4.0,930,"$2,000,000",$30,$30,NaN,11.0,1.0,24,2
1,tt0070913,Che?,Che?,1972,1972-12-08,Comedy,114,"Italy, France, West Germany","English, Italian, French",Roman Polanski,...,5.7,3256,<NA>,$64,<NA>,NaN,23.0,37.0,8,12
2,tt1986953,Storage 24,Storage 24,2012,2012-06-29,"Action, Horror, Mystery",87,UK,English,Johannes Roberts,...,4.4,6309,<NA>,$72,"$646,175",52.0,65.0,88.0,29,6
3,tt1865335,Confession of a Child of the Century,Confession of a Child of the Century,2012,2012-08-29,Drama,120,"France, Germany, UK",English,Sylvie Verheyde,...,4.4,514,EUR 4000000,$74,"$146,155",NaN,4.0,23.0,29,8
4,tt4195920,Chicas paranoicas,Chicas paranoicas,2016,2016-09-16,Comedy,100,Spain,Spanish,Pedro del Santo,...,8.0,169,<NA>,$78,$78,NaN,NaN,2.0,16,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9894098,Sathru,Sathru,2019,2019-03-07,Thriller,129,India,NaN,Naveen Nanjundan,...,6.1,163,<NA>,<NA>,"$8,683",NaN,7.0,1.0,7,3
81269,tt9899880,Columbus,Columbus,2018,2018-12-05,"Comedy, Drama",82,Iran,"Persian, English",Hatef Alimardani,...,4.0,130,<NA>,<NA>,<NA>,NaN,NaN,13.0,5,12
81270,tt9903716,Jessie,Jessie,2019,2019-03-15,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,7.2,219,<NA>,<NA>,<NA>,NaN,21.0,NaN,15,3
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,8.4,369,<NA>,<NA>,<NA>,NaN,NaN,NaN,8,3


In [9]:
# convert to numbers
raw_df['reviews_from_users'] = pd.to_numeric(raw_df["reviews_from_users"], downcast="integer")
raw_df['reviews_from_critics'] = pd.to_numeric(raw_df["reviews_from_critics"],  downcast="integer")
raw_df['web_rating'] = pd.to_numeric(raw_df["web_rating"],  downcast="integer")

In [10]:
# create a new col combining reviews from users and critics
raw_df['total_reviews'] = raw_df['reviews_from_users'] + raw_df['reviews_from_critics']

In [11]:
# # replace all NaN with empty
raw_df.fillna('', inplace=True)
raw_df

,id,title,original_title,year,date_published,genre,duration,country,language,director,...,vote_num,budget,usa_gross_income,worldwide_gross_income,web_rating,reviews_from_users,reviews_from_critics,day,month,total_reviews
0,tt0429277,Zyzzyx Rd,Zyzzyx Rd,2006,2006-02-24,"Crime, Drama, Thriller",90,USA,English,John Penney,...,930,"$2,000,000",$30,$30,,11,1,24,2,12
1,tt0070913,Che?,Che?,1972,1972-12-08,Comedy,114,"Italy, France, West Germany","English, Italian, French",Roman Polanski,...,3256,,$64,,,23,37,8,12,60
2,tt1986953,Storage 24,Storage 24,2012,2012-06-29,"Action, Horror, Mystery",87,UK,English,Johannes Roberts,...,6309,,$72,"$646,175",52,65,88,29,6,153
3,tt1865335,Confession of a Child of the Century,Confession of a Child of the Century,2012,2012-08-29,Drama,120,"France, Germany, UK",English,Sylvie Verheyde,...,514,EUR 4000000,$74,"$146,155",,4,23,29,8,27
4,tt4195920,Chicas paranoicas,Chicas paranoicas,2016,2016-09-16,Comedy,100,Spain,Spanish,Pedro del Santo,...,169,,$78,$78,,,2,16,9,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9894098,Sathru,Sathru,2019,2019-03-07,Thriller,129,India,,Naveen Nanjundan,...,163,,,"$8,683",,7,1,7,3,8
81269,tt9899880,Columbus,Columbus,2018,2018-12-05,"Comedy, Drama",82,Iran,"Persian, English",Hatef Alimardani,...,130,,,,,,13,5,12,
81270,tt9903716,Jessie,Jessie,2019,2019-03-15,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,219,,,,,21,,15,3,
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,369,,,,,,,8,3,


In [12]:
# print out all current column names
for col in raw_df.columns:
    print(f'"{col}",')

"id",
"title",
"original_title",
"year",
"date_published",
"genre",
"duration",
"country",
"language",
"director",
"writer",
"prod_co",
"actors",
"description",
"user_rating",
"vote_num",
"budget",
"usa_gross_income",
"worldwide_gross_income",
"web_rating",
"reviews_from_users",
"reviews_from_critics",
"day",
"month",
"total_reviews",


1. choose all cols to keep (netflix id, imdb id, title, ddmmyy, dd, mo,yr,duration, avg_vote, votes, reviews_from_users	reviews_from_critics, budget,usa_gross_icome, ww income, coutry, type, genre, actor, description)
[[""]]
2. break the date published col into ye, mo, dd
3. combine reviews
4. rename columns
5. load into SQL-pgadmin4
6. report
       a. where data found, how
       b. transforming process
       c. load SQL, pic of SQL

### DATABASE PROCESSED_DF

In [13]:
#  del processed_df

In [14]:
# # filter out un-needed columns and re-arrange columns
try:
    del processed_df

except Exception:
    pass

processed_df = raw_df #[[
# "id",
# "title",
# "day",
# "month",
# "year",
# "date_published",
# "user_rating",
# "web_rating",
# "vote_num",
# "reviews_from_users",
# "reviews_from_critics",
# "total_reviews",
# "budget",
# "usa_gross_income",
# "worldwide_gross_income",
# "duration",
# "country",
# "language",
# "genre",
# "director",
# "writer",
# "prod_co",
# "actors",
# "description"
# ]]

In [15]:
processed_df

,id,title,original_title,year,date_published,genre,duration,country,language,director,...,vote_num,budget,usa_gross_income,worldwide_gross_income,web_rating,reviews_from_users,reviews_from_critics,day,month,total_reviews
0,tt0429277,Zyzzyx Rd,Zyzzyx Rd,2006,2006-02-24,"Crime, Drama, Thriller",90,USA,English,John Penney,...,930,"$2,000,000",$30,$30,,11,1,24,2,12
1,tt0070913,Che?,Che?,1972,1972-12-08,Comedy,114,"Italy, France, West Germany","English, Italian, French",Roman Polanski,...,3256,,$64,,,23,37,8,12,60
2,tt1986953,Storage 24,Storage 24,2012,2012-06-29,"Action, Horror, Mystery",87,UK,English,Johannes Roberts,...,6309,,$72,"$646,175",52,65,88,29,6,153
3,tt1865335,Confession of a Child of the Century,Confession of a Child of the Century,2012,2012-08-29,Drama,120,"France, Germany, UK",English,Sylvie Verheyde,...,514,EUR 4000000,$74,"$146,155",,4,23,29,8,27
4,tt4195920,Chicas paranoicas,Chicas paranoicas,2016,2016-09-16,Comedy,100,Spain,Spanish,Pedro del Santo,...,169,,$78,$78,,,2,16,9,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9894098,Sathru,Sathru,2019,2019-03-07,Thriller,129,India,,Naveen Nanjundan,...,163,,,"$8,683",,7,1,7,3,8
81269,tt9899880,Columbus,Columbus,2018,2018-12-05,"Comedy, Drama",82,Iran,"Persian, English",Hatef Alimardani,...,130,,,,,,13,5,12,
81270,tt9903716,Jessie,Jessie,2019,2019-03-15,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,219,,,,,21,,15,3,
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,369,,,,,,,8,3,


In [16]:
# def convt_currency (df_name, col_name):
    
#     print (f">> Processing Column: '{col_name}'>>\n{50*('-')}")
    
#     # replace $ for USD and remove all "blanks"
#     df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace("$", "USD ")
#     df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace(",", "")

#     # split currency and value into two cols
#     for row in df_name[col_name]:
#         try:
#             new_col = f'{col_name}_currency'
# #             print(new_col)
#             df_name[new_col], df_name[col_name] = df_name[col_name].str.split(' ', 1)

#         except ValueError:
#             pass
    
#     df_name.fillna('', inplace=True)
        
#     return print(">> DONE!")

In [17]:
processed_df[['worldwide_gross_income', 'usa_gross_income', 'budget']]

,worldwide_gross_income,usa_gross_income,budget
0,$30,$30,"$2,000,000"
1,,$64,
2,"$646,175",$72,
3,"$146,155",$74,EUR 4000000
4,$78,$78,
...,...,...,...
81268,"$8,683",,
81269,,,
81270,,,
81271,,,


In [18]:
def conv_currency (df_name, col_name):
    
    print (f"\n>> Processing Column: '{col_name}'")
    
    # replace $ for USD and remove all "blanks"
    df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace("$", "USD ")
    df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace(",", "")

    # split currency and value into two cols
    
    try:
        new_col = f'{col_name}_currency'
        print(f'\t>>> Creating new column named: "{new_col}"')
        df_name[new_col], df_name[col_name] = df_name[col_name].str.split(' ', 1).str

    except ValueError:
        pass
    
    df_name.fillna('', inplace=True)
    
    return print(f" >> DONE!\n\n{50*('=')}\n{50*('=')}")

In [19]:
col_to_covert = ['worldwide_gross_income', 'usa_gross_income', 'budget']

In [20]:
for ea_col in col_to_covert:
    conv_currency(processed_df, ea_col)
processed_df[['worldwide_gross_income', 'usa_gross_income', 'budget']]



>> Processing Column: 'worldwide_gross_income'
	>>> Creating new column named: "worldwide_gross_income_currency"


C:\Users\diamo\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


 >> DONE!


>> Processing Column: 'usa_gross_income'
	>>> Creating new column named: "usa_gross_income_currency"
 >> DONE!


>> Processing Column: 'budget'
	>>> Creating new column named: "budget_currency"
 >> DONE!



,worldwide_gross_income,usa_gross_income,budget
0,30,30,2000000
1,,64,
2,646175,72,
3,146155,74,4000000
4,78,78,
...,...,...,...
81268,8683,,
81269,,,
81270,,,
81271,,,


In [21]:
!mkdir archieve
processed_df[["worldwide_gross_income_currency", 'worldwide_gross_income', "usa_gross_income_currency", 'usa_gross_income', "budget_currency", 'budget']].to_csv("archieve\exported_draft.csv")

A subdirectory or file archieve already exists.


In [22]:
for col in processed_df.columns:
    print (f'"{col}",')

"id",
"title",
"original_title",
"year",
"date_published",
"genre",
"duration",
"country",
"language",
"director",
"writer",
"prod_co",
"actors",
"description",
"user_rating",
"vote_num",
"budget",
"usa_gross_income",
"worldwide_gross_income",
"web_rating",
"reviews_from_users",
"reviews_from_critics",
"day",
"month",
"total_reviews",
"worldwide_gross_income_currency",
"usa_gross_income_currency",
"budget_currency",


In [23]:
print(raw_df.dtypes)

id                                         object
title                                      object
original_title                             object
year                                        int64
date_published                     datetime64[ns]
genre                                      object
duration                                    int64
country                                    object
language                                   object
director                                   object
writer                                     object
prod_co                                    object
actors                                     object
description                                object
user_rating                               float64
vote_num                                    int64
budget                                     object
usa_gross_income                           object
worldwide_gross_income                     object
web_rating                                 object


In [24]:
cols=[
"id",
"title",

"date_published",
"day",
"month",
"year",

"user_rating",
"web_rating",

"vote_num",
"reviews_from_users",
"reviews_from_critics",
"total_reviews",



"usa_gross_income_currency",
"usa_gross_income",

"worldwide_gross_income_currency",
"worldwide_gross_income",

"budget_currency",
"budget",


"duration",
"country",
"language",
"genre",
"director",
"writer",
"prod_co",
"actors",
"description"
]

processed_df = processed_df[cols]

In [25]:
processed_df

,id,title,date_published,day,month,year,user_rating,web_rating,vote_num,reviews_from_users,...,budget,duration,country,language,genre,director,writer,prod_co,actors,description
0,tt0429277,Zyzzyx Rd,2006-02-24,24,2,2006,4.0,,930,11,...,2000000,90,USA,English,"Crime, Drama, Thriller",John Penney,John Penney,Zyzzyx LLC,"Leo Grillo, Katherine Heigl, Tom Sizemore, Ric...",The family man accountant Grant travels to Las...
1,tt0070913,Che?,1972-12-08,8,12,1972,5.7,,3256,23,...,,114,"Italy, France, West Germany","English, Italian, French",Comedy,Roman Polanski,"Gérard Brach, Roman Polanski",Compagnia Cinematografica Champion,"Marcello Mastroianni, Sydne Rome, Hugh Griffit...","During her Italian vacation, a young and beaut..."
2,tt1986953,Storage 24,2012-06-29,29,6,2012,4.4,52,6309,65,...,,87,UK,English,"Action, Horror, Mystery",Johannes Roberts,"Noel Clarke, Noel Clarke",Medient Studios,"Noel Clarke, Colin O'Donoghue, Antonia Campbel...","In London, a military plane crashes leaving it..."
3,tt1865335,Confession of a Child of the Century,2012-08-29,29,8,2012,4.4,,514,4,...,4000000,120,"France, Germany, UK",English,Drama,Sylvie Verheyde,"Alfred de Musset, Sylvie Verheyde",Les Films du Veyrier,"Charlotte Gainsbourg, Pete Doherty, August Die...","Paris, 1830: Octave, betrayed by his mistress,..."
4,tt4195920,Chicas paranoicas,2016-09-16,16,9,2016,8.0,,169,,...,,100,Spain,Spanish,Comedy,Pedro del Santo,"Helena Montesinos, Helena Montesinos",,"Patricia Valley, Mairen Muñoz, Marta Mir Martí...",'Chicas Paranoicas' is the first Spanish comed...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9894098,Sathru,2019-03-07,7,3,2019,6.1,,163,7,...,,129,India,,Thriller,Naveen Nanjundan,,Milestone Movies,"Srushti Dange, Kathir, Laguparan, Marimuthu, N...",A kidnapping gone wrong leads to mounting tens...
81269,tt9899880,Columbus,2018-12-05,5,12,2018,4.0,,130,,...,,82,Iran,"Persian, English","Comedy, Drama",Hatef Alimardani,Hatef Alimardani,,"Farhad Aslani, Majid Salehi, Saeed Poursamimi,...",A rich family are deciding to immigrate to the...
81270,tt9903716,Jessie,2019-03-15,15,3,2019,7.2,,219,21,...,,106,India,Telugu,"Horror, Thriller",Aswani Kumar V.,Aswani Kumar V.,Ekaa Art Production,"Sritha Chandana, Pavani Gangireddy, Abhinav Go...","Set in an abandoned house, the film follows a ..."
81271,tt9911774,Padmavyuhathile Abhimanyu,2019-03-08,8,3,2019,8.4,,369,,...,,130,India,Malayalam,Drama,Vineesh Aaradya,"Vineesh Aaradya, Vineesh Aaradya",RMCC Productions,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",


In [26]:
for col in  cols:
    print (f'{col} = Column(    )')

id = Column(    )
title = Column(    )
date_published = Column(    )
day = Column(    )
month = Column(    )
year = Column(    )
user_rating = Column(    )
web_rating = Column(    )
vote_num = Column(    )
reviews_from_users = Column(    )
reviews_from_critics = Column(    )
total_reviews = Column(    )
usa_gross_income_currency = Column(    )
usa_gross_income = Column(    )
worldwide_gross_income_currency = Column(    )
worldwide_gross_income = Column(    )
budget_currency = Column(    )
budget = Column(    )
duration = Column(    )
country = Column(    )
language = Column(    )
genre = Column(    )
director = Column(    )
writer = Column(    )
prod_co = Column(    )
actors = Column(    )
description = Column(    )


### SQL-Alchemy to load clean data from pandas DataFrame to SQL Lite DataBase

In [27]:
# create declarative base
Base = declarative_base()

# check current table available in the Base - should be nothing at this point
Base.metadata.tables


immutabledict({})

In [28]:
# folder name that will store the sql-lite database
fol_name = "Exp_SqlLiteDb"
if pathlib.Path(fol_name).exists():
    print(f' >> Folder "{fol_name}" already exists!\n >> No new folder was created ...')
    pass
else:
    ! mkdir Exp_SqlLiteDb
    print(f'Successfully created folder "{fol_name}"')

 >> Folder "Exp_SqlLiteDb" already exists!
 >> No new folder was created ...


In [29]:
# # folder name that will store the sql-lite database
# database_name = "Exp_SqlLiteDb"
# if pathlib.Path(fol_name).exists():
#     print(f' >> Folder "{fol_name}" already exists!\n >> No new folder was created ...')
#     pass
# else:
#     ! mkdir Exp_SqlLiteDb
#     print(f'Successfully created folder "{fol_name}"')

In [30]:
sqlLite_db_path = "Exp_SqlLiteDb\movie.db"
engine = create_engine(f"sqlite:///{sqlLite_db_path}")
conn = engine.connect()

In [31]:
# print out list of cols for class creation
i=0
for col in processed_df.columns:
    print(f'{col} = Column()')
    i+=1
print(f'\n{("-")*50}\n>> There are total {i} columns in the current data frame.')

id = Column()
title = Column()
date_published = Column()
day = Column()
month = Column()
year = Column()
user_rating = Column()
web_rating = Column()
vote_num = Column()
reviews_from_users = Column()
reviews_from_critics = Column()
total_reviews = Column()
usa_gross_income_currency = Column()
usa_gross_income = Column()
worldwide_gross_income_currency = Column()
worldwide_gross_income = Column()
budget_currency = Column()
budget = Column()
duration = Column()
country = Column()
language = Column()
genre = Column()
director = Column()
writer = Column()
prod_co = Column()
actors = Column()
description = Column()

--------------------------------------------------
>> There are total 27 columns in the current data frame.


In [32]:
# column types in pd_df
processed_df.dtypes

id                                         object
title                                      object
date_published                     datetime64[ns]
day                                         int64
month                                       int64
year                                        int64
user_rating                               float64
web_rating                                 object
vote_num                                    int64
reviews_from_users                         object
reviews_from_critics                       object
total_reviews                              object
usa_gross_income_currency                  object
usa_gross_income                           object
worldwide_gross_income_currency            object
worldwide_gross_income                     object
budget_currency                            object
budget                                     object
duration                                    int64
country                                    object


id                               object
title                            object
day                               int64
month                             int64
year                              int64
date_published           datetime64[ns]
user_rating                     float64
web_rating                      float64
vote_num                          int64
reviews_from_users              float64
reviews_from_critics            float64
total_reviews                   float64
budget                           object
usa_gross_income                 object
worlwide_gross_income            object
duration                          int64
country                          object
language                         object
genre                            object
director                         object
writer                           object
prod co                          object
actors                           object
description                      object

In [ ]:
class Movie2(Base):
    
    table_name = input("Please name your table.")
    
    __tablename__ = table_name
    
    id = Column(Integer, primary_key=True)
    title = Column(String(255)
                   
    date_published = Column(String(20))
                   
    day = Column(Integer)
    month = Column(Integer)
    year = Column(Integer)
                   
    user_rating = Column(    )
    web_rating = Column(    )
    vote_num = Column(    )
    reviews_from_users = Column(    )
    reviews_from_critics = Column(    )
    total_reviews = Column(    )
    
    usa_gross_income_currency = Column(    )
    usa_gross_income = Column(Integer)
    
    worldwide_gross_income_currency = Column(    )
    worldwide_gross_income = Column(Integer)
   
    budget_currency = Column(Integer)
    budget = Column(Integer)
       
    duration = Column(Integer)
                   
    country = Column(String(50))
    language = Column(String(50))
    genre = Column(String(50))
    director = Column(String(50))
    writer = Column(String(50))
    prod_co = Column(String(50))
    actors = Column(String(50))
    description = Column(String(500)String(50))

In [ ]:
class Movie2(Base):
    table_name = input("Please name your table.")
    __tablename__ = table_name
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    day = Column(Integer)
    month = Column(Integer)
    year = Column(Integer)
    date_published = Column(String(15))
    user_rating = Column(REAL)
    web_rating = Column(REAL)
    vote_num = Column(Integer)
    reviews_from_users = Column(Integer)
    reviews_from_critics = Column(Integer)
    total_reviews = Column(Integer)
    budget = Column(String(255))
    usa_gross_income = Column(String(255))
    worlwide_gross_income = Column(Interger)
    duration = Column(Interger)
    country = Column(String(50)
    language = Column(String(50))
    genre = Column(String(50))
    director = Column(String(50))
    writer = Column(String(50))
    prod_co = Column(String(50))
    actors = Column(String(50))
    description = Column(String(500))


In [ ]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

In [ ]:
# current in memory tables
Base.metadata.tables

In [ ]:
# orm requires session so rollbacks can occur etc.
session = Session(bind=engine)

In [ ]:
# print out all current column names
for col in processed_df.columns:
    print(f'{col} = ,')

In [ ]:
test = Movie(
id = 34364256,
title = "testing while running",
day = 25,
month = 4,
year = 2020,
date_published = 9-5-2001 ,
user_rating = 5.7 ,
web_rating = 7.6,
vote_num = 345,
reviews_from_users = 456,
reviews_from_critics = 234,
total_reviews = 836,
budget = 987654321,
usa_gross_income = 123456789,
worlwide_gross_income = 1234567891011,
duration = 70,
country = 'USA',
language = "American",
genre = "thriller",
director = " Mr. Jon Dhoe",
writer = "warming",
prod_co = "Universal",
actors = "Brad Pitt, Olando Bloom",
description = "This is a test movie just to make sure all is running well")

In [ ]:
# for index, row in processed_df.iterrows():
#     row['id'] = row['id'].replace("tt", "") 
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace("$", "")
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace(" ", "")
# processed_df

In [ ]:
# add one item
session.add(patterson)

In [ ]:
session.commit()

In [ ]:
session.commit()
engine.execute("select * from cat").fetchall()

In [ ]:
session.close()